In [1]:
from binary_param import linear, constant
import pandas as pd
import numpy as np
from R32_bmimPF6_PR import configuration
from pyomo.environ import (ConcreteModel,
                           SolverFactory,
                           units as pyunits)

import idaes.logger as idaeslog
import pyomo.contrib.parmest.parmest as parmest

import pytest

from idaes.core import FlowsheetBlock
from idaes.generic_models.unit_models import Flash
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

from idaes.core.util.model_statistics import degrees_of_freedom

In [2]:
def PR_model(data):
    """
    Function which returns initilized model.

    Args:
        data: pandas DataFrame with data

    Returns:
        initialized model
    """
    eps = 0.1
    init_kappa_2_1A = 0.63257
    init_kappa_1_2A = 0.06399
    init_kappa_2_1B = -0.69149
    init_kappa_1_2B = -0.10862

   
    m = ConcreteModel()

    m.fs = FlowsheetBlock(default={"dynamic": False})

    m.fs.properties = GenericParameterBlock(default=configuration)

    m.fs.state_block = m.fs.properties.state_block_class(
        default={"parameters": m.fs.properties,
                 "defined_state": True})
    x = float(data['x_R32'])+eps
    m.fs.state_block.flow_mol.fix(1)
    m.fs.state_block.temperature.fix(float(data["temperature"]))
    m.fs.state_block.pressure.fix(float(data["pressure"]))
    m.fs.state_block.mole_frac_comp['bmimPF6'].fix(1-x)
    m.fs.state_block.mole_frac_comp['R32'].fix(x)

    # parameter - kappa_A_ij (set at 0.3, 0 if i=j)
    m.fs.properties.PR_kappa_A['bmimPF6','bmimPF6'].fix(0)
    m.fs.properties.PR_kappa_A['bmimPF6', 'R32'].fix(init_kappa_2_1A)
    m.fs.properties.PR_kappa_A['R32', 'R32'].fix(0)
    m.fs.properties.PR_kappa_A['R32', 'bmimPF6'].fix(init_kappa_1_2A)
    m.fs.properties.PR_kappa_B['bmimPF6', 'bmimPF6'].fix(0)
    m.fs.properties.PR_kappa_B['bmimPF6', 'R32'].fix(init_kappa_2_1B)
    m.fs.properties.PR_kappa_B['R32', 'R32'].fix(0)
    m.fs.properties.PR_kappa_B['R32', 'bmimPF6'].fix(init_kappa_1_2B)
    m.fs.properties.PR_kappa_C['bmimPF6', 'bmimPF6'].fix(0)
    m.fs.properties.PR_kappa_C['bmimPF6', 'R32'].fix(0)
    m.fs.properties.PR_kappa_C['R32', 'R32'].fix(0)
    m.fs.properties.PR_kappa_C['R32', 'bmimPF6'].fix(0)
    # Initialize the flash unit
    m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

    # Fix the state variables on the state block
    m.fs.state_block.pressure.unfix()
    m.fs.state_block.mole_frac_comp['bmimPF6'].unfix()
    m.fs.state_block.mole_frac_comp['R32'].unfix()
    m.fs.state_block.temperature.fix(float(data["temperature"]))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R32'].fix(float(data['x_R32']))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'bmimPF6'].fix(float(data['x_bmimPF6']))
    m.fs.state_block.mole_frac_comp['R32'].fix(float(data['x_R32'])+eps)
    m.fs.state_block.mole_frac_comp['bmimPF6'].unfix()

    # Set bounds on variables to be estimated
    m.fs.properties.PR_kappa_A['bmimPF6', 'R32'].setlb(-5)
    m.fs.properties.PR_kappa_A['bmimPF6', 'R32'].setub(5)

    m.fs.properties.PR_kappa_A['R32', 'bmimPF6'].setlb(-5)
    m.fs.properties.PR_kappa_A['R32', 'bmimPF6'].setub(5)

    # Return initialized flash model
    return m

def SSE(m, data):
    """
    returns objective function expresion.

    Args:
        m: model
        data: pandas DataFrame with data

    Returns:
        objectuve function scaled expresion
    """
    expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)

    return expr * 1e-6

variable_name = ["fs.properties.PR_kappa_A['bmimPF6', 'R32']",
                 "fs.properties.PR_kappa_A['R32', 'bmimPF6']",
                 "fs.properties.PR_kappa_B['bmimPF6', 'R32']",
                 "fs.properties.PR_kappa_B['R32', 'bmimPF6']"]

#     pest = parmest.Estimator(PR_model, data, variable_name, SSE, tee=True)

#     obj_value, parameters,a= pest.theta_est(calc_cov=True)

In [3]:
data = pd.read_csv('R32IL.csv')

# print(data)


In [4]:
N = 10 #number of samples
results_params = np.zeros((N,4))
results_obj = np.zeros((N,1))

st_dev_T = 0.01 #C
st_dev_P = 9771 #MPa
st_dev_x = 0.10 #mole fraction

for i in range(N):
    print(i)
#     noise_T = np.random.normal(loc=0,scale=st_dev_T)
#     print(noise_T)
#     noise_P = np.random.normal(loc=0,scale=st_dev_P)
#     print(noise_P)
    noise_x = np.random.normal(loc=0,scale=st_dev_x)
    print(noise_x)
    data_ = data.copy()
#     data_["temperature"] += noise_T
    data_["x_bmimPF6"] += data_["x_bmimPF6"] * noise_x
#     data_["pressure"] += noise_P
    
#     data_.loc[data_['pressure'] <= 0, 'pressure'] = 1000
    data_.loc[data_['x_bmimPF6'] <= 0.0001, 'x_bmimPF6'] = 0.0001
    data_.loc[data_['x_bmimPF6'] >= 0.9999 , 'x_bmimPF6'] = 0.9999      

    try:
        pest = parmest.Estimator(PR_model, data_, variable_name, SSE, tee=False)

        obj_value, parameters = pest.theta_est()

        for k,v in parameters.items():
            print(k, "=", v)

        line = 0

        for k,v in parameters.items():
            results_params[i,line] = v
            line += 1
        results_obj[i] = obj_value

    except ValueError:
        results_params[i,0] = 'NaN'
        results_params[i,1] = 'NaN'


0
0.10770261392034451
2021-12-30 12:02:21 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 12:02:21 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 12:02:22 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 12:02:23 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 12:02:24 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-12-30 12:02:25 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-12-30 12:02:25 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 12:02:26 [INFO] idaes.in

2021-12-30 12:03:21 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 12:03:22 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 12:03:23 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 12:03:23 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 12:03:24 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-12-30 12:03:25 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-12-30 12:03:26 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 12:03:26 [INFO] idaes.init.fs.state_block: Pro

2021-12-30 12:04:19 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 12:04:20 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 12:04:21 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 12:04:21 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 12:04:22 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-12-30 12:04:23 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-12-30 12:04:24 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 12:04:24 [INFO] idaes.init.fs.state_block: Pro

2021-12-30 12:05:17 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 12:05:18 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 12:05:18 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 12:05:19 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 12:05:20 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-12-30 12:05:21 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-12-30 12:05:21 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 12:05:22 [INFO] idaes.init.fs.state_block: Pro

2021-12-30 12:06:19 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 12:06:20 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 12:06:21 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 12:06:21 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 12:06:23 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-12-30 12:06:24 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-12-30 12:06:25 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 12:06:26 [INFO] idaes.init.fs.state_block: Pro

In [5]:
print(results_params)

[[ 2.80998133e-01  4.94764623e-02 -3.08757538e-01 -8.50980218e-02]
 [ 2.96810480e-01  5.84576863e-02 -3.38507015e-01 -1.01278891e-01]
 [ 2.81139397e-01  4.95502601e-02 -3.09025746e-01 -8.52349905e-02]
 [ 3.13437680e-01  4.86965033e-02 -3.25946678e-01 -7.61459297e-02]
 [-2.10062281e+00  4.85255815e-02  2.22765597e+00 -9.09164684e-02]
 [-4.79165362e+00 -2.47845046e-02  5.10906357e+00 -2.52585960e-04]
 [ 8.43790206e-01  7.52388460e-02 -9.27472077e-01 -1.29243457e-01]
 [ 3.34600980e-01  6.25424328e-02 -3.82188912e-01 -1.08001728e-01]
 [-2.56627355e+00  4.48158988e-02  2.76612388e+00 -7.91924045e-02]
 [ 2.86256832e-01  4.64230439e-02 -3.03965363e-01 -7.67646948e-02]]


In [6]:
results = pd.DataFrame(results_params)
results.to_csv('R32_bmim_10_T.csv')